# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
pd_data['class_buys_computer'].value_counts()

yes    9
no     5
Name: class_buys_computer, dtype: int64

In [4]:
# 분자는 타깃변수의 정답개수
# 분모는 Di의 전체갯수

def get_gini(df, label):
    labelClassCount = df[label].value_counts() # Label의 class별 count
    eachClassCount = labelClassCount.values #클래스별 개수 
    allCount = len(df[label])  #전체 개수
    
    # gini계수 구하기
    # 1 - sum( (클래스별개수/전체개수)의 제곱 )
    gini = 1 - sum([(count/allCount)**2 for count in eachClassCount])
    
    return gini

In [5]:
get_gini(pd_data,'class_buys_computer') #0.4591836734693877

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [6]:
def get_binary_split(df, attribute):
    result = list()
    attrList = list(df[attribute].unique()) # 각 attribute구하기
    for eachAttribute in attrList: # 각 attribute를 첫번째 set으로 놓기
        index = attrList.index(eachAttribute) # set 구분하기 위한 index
        result.append([[eachAttribute],
                        attrList[:index] + attrList[index+1:]]) # 첫번째 set 제외하고 나머지를 otherset으로 두어 결과에 추가
        
    return result

In [7]:
get_binary_split(pd_data,"age")

[[['youth'], ['middle_aged', 'senior']],
 [['middle_aged'], ['youth', 'senior']],
 [['senior'], ['youth', 'middle_aged']]]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [8]:
def get_attribute_gini_index(df, attribute, label):

    binarySet = get_binary_split(df, attribute)
    allCount = len(df) # 전체 레코드 개수 
    result = dict()
    
    for _ in binarySet:
        # 나눠진 셋 [['youth'], ['middle_aged', 'senior']]
        for eachSet in _:
            # 각각의 셋 ex) ['youth'], ['middle_aged', 'senior']

            giniDf = pd.DataFrame() # concat할 임시 데이터 프레임
            for attrName in eachSet:
                tempDf = df[df[attribute] == attrName] # 
                giniDf = pd.concat([giniDf, tempDf])
            
            df1 = giniDf # attrName 셋
            df2 = pd_data.drop(giniDf.index) # 나머지 셋
            
            gini1 = get_gini(df1, label) # 첫번째 셋 지니계수 구하기
            gini2 = get_gini(df2, label) # 나머지 지니계수 구하기
            
            key = (' & ').join(eachSet) # key이름 만들기 ex) youth & middle_aged
            
            # set1의개수/전체개수 * gini(d1) + set2의개수/전체개수 * gini(d2)... 
            result[key] = (len(df1)/allCount * gini1) + (len(df2)/allCount * gini2)

    return result

In [9]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged & senior': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'youth & senior': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth & middle_aged': 0.45714285714285713}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [10]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())
# ('middle_aged', 0.35714285714285715)

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [11]:
##문제1 답안
get_binary_split(pd_data, "income")

[[['high'], ['medium', 'low']],
 [['medium'], ['high', 'low']],
 [['low'], ['high', 'medium']]]

In [12]:
##문제2 답안

# 분류를 하는 데 가장 중요한 변수 : gini index가 가장 낮은 feature
featureGini = dict()

for column in pd_data.columns[:-1]: #'class_buys_computer'빼고 모든 feature의 지니인덱스 구하기
    gini = min(get_attribute_gini_index(pd_data, column, "class_buys_computer").items())
    featureGini[column] = gini # key = column명, value = ('set', gini_index)
    
print(sorted(featureGini.values(), key=lambda x:x[1])[0])
featureGini

('middle_aged', 0.35714285714285715)


{'age': ('middle_aged', 0.35714285714285715),
 'income': ('high', 0.4428571428571429),
 'student': ('no', 0.3673469387755103),
 'credit_rating': ('excellent', 0.42857142857142855)}

In [13]:
##문제3 답안
splitKey = list(featureGini.keys())[0] # gini_index가 제일 낮은 feature

# feature의 set기준으로 dataframe분리
df1 = pd_data[pd_data[splitKey] == featureGini[splitKey][0]] 
df2 = pd_data[pd_data[splitKey] != featureGini[splitKey][0]]

# columnList에서 label하고 splitKey 제거
columnList = list(pd_data.columns[:-1])
columnList.remove(splitKey)

featureGini1 = dict()

# middle_age로 분리된 df1
for column in columnList:
    gini = min(get_attribute_gini_index(df1, column, "class_buys_computer").items())
    featureGini1[column] = gini
print("df1 split feature :",
      list(featureGini1.keys())[0],
      sorted(featureGini1.values(), key=lambda x:x[1])[0])

# youth & senior로 분리된 df2
# 얘는 age가 2클래스가 남아있기 때문에 age의 gini_index도 구해줘야 한다.
featureGini2 = dict()
for column in pd_data.columns[:-1]:
    gini = min(get_attribute_gini_index(df2, column, "class_buys_computer").items())
    featureGini2[column] = gini
print("df2 split feature :",
      list(featureGini2.keys())[0],
      sorted(featureGini2.values(), key=lambda x:x[1])[0])

df1 split feature : income ('high', 1.458333333333333)
df2 split feature : age ('no', 0.33777777777777773)


# DT Assignment2
# ID3 Algorithm

In [14]:
from math import log2

def get_entropy(df, label):
    labelClassCount = df[label].value_counts() # Label의 class별 count
    eachClassCount = labelClassCount.values #클래스별 개수 
    allCount = len(df[label])  #전체 개수
    
    # entropy 구하기
    # -label개수/전체개수 x log2(label개수/전체개수)의 합이다.
    entropy = sum([-(count/allCount) * log2(count/allCount) for count in eachClassCount])
    
    return entropy

In [15]:
# 전체 Information 구하기
# info(D) = entropy(label)
infoD = get_entropy(pd_data, 'class_buys_computer')
infoD

0.9402859586706311

In [21]:
# 각 feature의 Information Gain 구하기
def featureInfo(df, feature, label):
    
    labelClassCount = df[feature].value_counts() # feature의 class별 count
    eachClassCount = labelClassCount.values #클래스별 개수 
    allCount = len(df[feature])  #전체 개수
    
    infoA = 0
    for i, element in enumerate(df[feature].unique()):
        # class개수/전체개수 * 엔트로피
        infoA += eachClassCount[i]/allCount \
                * get_entropy(df[df[feature] == element], label)
    
    return infoD - infoA #전체IG - featurIG
featureInfo(pd_data, 'age', 'class_buys_computer')

0.31610343366405846

In [22]:
IG_Dict = dict()
for column in pd_data.columns[:-1]:
    IG_Dict[column] = featureInfo(pd_data, column, 'class_buys_computer')
sorted(IG_Dict.items(), key=lambda x:x[1], reverse=True)[0]

('age', 0.31610343366405846)